In [1]:
import keras
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense,Bidirectional
from nltk.tokenize import word_tokenize,sent_tokenize
from keras.layers import *
from sklearn.model_selection import cross_val_score 
import nltk
import pandas as pd
nltk.download('punkt')

2024-02-21 15:26:04.112186: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/smridhibhat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df = pd.read_csv('isear.csv',header = None)

In [3]:
# The isear.csv contains rows with value 'No response'
# We need to remove such rows
df.drop(df[df[1] == '[ No response.]'].index, inplace = True)
print(df.head())

         0                                                  1
0      joy  [ On days when I feel close to my partner and ...
1     fear  Every time I imagine that someone I love or I ...
2    anger  When I had been obviously unjustly treated and...
3  sadness  When I think about the short time that we live...
4  disgust  At a gathering I found myself involuntarily si...


In [4]:
# The feel_arr will store all the sentences
# i.e feel_arr is the list of all sentences
feel_arr = df[1]

In [5]:
# Each  sentence in feel_arr is tokenized by the help of work tokenizer.
# If I have a sentence - 'I am happy'. 
# After word tokenizing it will convert into- ['I','am','happy']
feel_arr = [word_tokenize(sent) for sent in feel_arr]
print(feel_arr[0])

['[', 'On', 'days', 'when', 'I', 'feel', 'close', 'to', 'my', 'partner', 'and', 'other', 'friends', '.', 'When', 'I', 'feel', 'at', 'peace', 'with', 'myself', 'and', 'also', 'experience', 'a', 'close', 'contact', 'with', 'people', 'whom', 'I', 'regard', 'greatly', '.', ']']


In [6]:
# Defined a function padd in which each sentence length
# is fixed to 100.
# If length is less than 100 , then 
# the word- '<padd>' is append
def padd(arr):
    for i in range(100-len(arr)):
        arr.append('<pad>')
    return arr[:100]
  
# call the padd function for each sentence in feel_arr
for i in range(len(feel_arr)):
    feel_arr[i]=padd(feel_arr[i])

In [7]:
print(feel_arr[0])

['[', 'On', 'days', 'when', 'I', 'feel', 'close', 'to', 'my', 'partner', 'and', 'other', 'friends', '.', 'When', 'I', 'feel', 'at', 'peace', 'with', 'myself', 'and', 'also', 'experience', 'a', 'close', 'contact', 'with', 'people', 'whom', 'I', 'regard', 'greatly', '.', ']', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [8]:
# Glove vector contains a 50 dimensional vector corresponding 
# to each word in dictionary.
vocab_f = 'glove.6B.50d.txt'

In [9]:
# embeddings_index is a dictionary which contains the mapping of
# word with its corresponding 50d vector.
embeddings_index = {}
with open(vocab_f,encoding='utf8') as f:
    for line in f:
        # splitting each line of the glove.6B.50d in a list of 
        # items- in which the first element is the word to be embedded,
        # and from second to the end of line contains the 50d vector.
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [10]:
# the embedding index of word 'happy'
print( embeddings_index['happy'])

[ 0.092086  0.2571   -0.58693  -0.37029   1.0828   -0.55466  -0.78142
  0.58696  -0.58714   0.46318  -0.11267   0.2606   -0.26928  -0.072466
  1.247     0.30571   0.56731   0.30509  -0.050312 -0.64443  -0.54513
  0.86429   0.20914   0.56334   1.1228   -1.0516   -0.78105   0.29656
  0.7261   -0.61392   2.4225    1.0142   -0.17753   0.4147   -0.12966
 -0.47064   0.3807    0.16309  -0.323    -0.77899  -0.42473  -0.30826
 -0.42242   0.055069  0.38267   0.037415 -0.4302   -0.39442   0.10511
  0.87286 ]


In [11]:
# Embedding each word of the feel_arr
embedded_feel_arr=[] 
for each_sentence in feel_arr:
    embedded_feel_arr.append([])
    for word in each_sentence:
        if word.lower() in embeddings_index:
            embedded_feel_arr[-1].append(embeddings_index[word.lower()])
        else:
            # if the word to be embedded is '<padd>' append 0 fifty times
            embedded_feel_arr[-1].append([0]*50)

In [12]:
print(embedded_feel_arr[0][0])

[-0.61201   0.98226   0.11539   0.014623  0.23873  -0.067035  0.30632
 -0.64742  -0.38517  -0.03691   0.094788  0.57631  -0.091557 -0.54825
  0.25255  -0.14759   0.13023   0.21658  -0.30623   0.30028  -0.23471
 -0.17927   0.9518    0.54258   0.31172  -0.51038  -0.65223  -0.48858
  0.13486  -0.40132   2.493    -0.38777  -0.26456  -0.49414  -0.3871
 -0.20983   0.82941  -0.46253   0.39549   0.014881  0.79485  -0.79958
 -0.16243   0.013862 -0.53536   0.52536   0.019818 -0.16353   0.30649
  0.81745 ]


In [13]:
#Converting x into numpy-array
X = np.array(embedded_feel_arr)
print(np.shape(X))

(7575, 100, 50)


In [14]:
# Perform one-hot encoding on df[0] i.e emotion
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
Y = enc.fit_transform(np.array(df[0]).reshape(-1,1)).toarray()

# Split into train and test
from keras.layers import Embedding
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#Defining the BiLSTM Model
def model(X,Y,input_size1,input_size2,output_size):
    m = Sequential()
    m.add(Bidirectional(LSTM(100,input_shape=(input_size1,input_size2))))
    m.add(Dropout(0.5))
    m.add(Dense(output_size,activation='softmax'))
    m.compile('Adam','categorical_crossentropy',['accuracy'])
    m.fit(X,Y,epochs=32, batch_size=128)
    return m

In [ ]:
# calling the model function with the traning data inorder to build the model
bilstmModel = model(X_train,Y_train,100,50,7)

Epoch 1/32
48/48 [==============================] - 27s 481ms/step - loss: 1.8895 - accuracy: 0.2165
Epoch 2/32
48/48 [==============================] - 23s 486ms/step - loss: 1.7157 - accuracy: 0.3343
Epoch 3/32
48/48 [==============================] - 26s 538ms/step - loss: 1.6271 - accuracy: 0.3767
Epoch 4/32
48/48 [==============================] - 27s 564ms/step - loss: 1.5769 - accuracy: 0.3965
Epoch 5/32
48/48 [==============================] - 29s 591ms/step - loss: 1.5368 - accuracy: 0.4144
Epoch 6/32
48/48 [==============================] - 25s 529ms/step - loss: 1.4960 - accuracy: 0.4388
Epoch 7/32
48/48 [==============================] - 27s 567ms/step - loss: 1.4740 - accuracy: 0.4485
Epoch 8/32
48/48 [==============================] - 40s 833ms/step - loss: 1.4500 - accuracy: 0.4625
Epoch 9/32
48/48 [==============================] - 47s 968ms/step - loss: 1.4453 - accuracy: 0.4571
Epoch 10/32
48/48 [==============================] - 53s 1s/step - loss: 1.3760 - accuracy:

In [50]:
# print the summary of the model
bilstmModel.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_3 (Bidirecti  (None, 200)               120800    
 onal)                                                           
                                                                 
 dropout_3 (Dropout)         (None, 200)               0         
                                                                 
 dense_3 (Dense)             (None, 7)                 1407      
                                                                 
Total params: 122207 (477.37 KB)
Trainable params: 122207 (477.37 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [51]:
 #Testing the model
bilstmModel.evaluate(X_test,Y_test)

48/48 [==============================] - 2s 19ms/step - loss: 1.5037 - accuracy: 0.5122


[1.5037239789962769, 0.5122112035751343]

In [ ]:
# Checking the results of our model with user given sentences stored in test.csv file

df_test = pd.read_csv('test.csv',header = None)
df_test.drop(df[df[1] == '[ No response.]'].index, inplace = True)
feel_arr_test=df_test[1]


feel_arr_test=[word_tokenize(sent) for sent in feel_arr_test]

for i in range(len(feel_arr1)):
    feel_arr_test[i]=padd(feel_arr_test[i])

embedded_feel_arr_test=[] 
for each_sentence in feel_arr_test:
    embedded_feel_arr_test.append([])
    for word in each_sentence:
        if word.lower() in embeddings_index:
            embedded_feel_arr_test[-1].append(embeddings_index[word.lower()])
        else:
            embedded_feel_arr_test[-1].append([0]*50)
            
X_test = np.array(embedded_feel_arr_test)

In [73]:
prediction = bilstmModel.predict(X_test)

1/1 [==============================] - 0s 98ms/step


In [97]:
# sentence: 'Grovelling people.' , Emotion: Digust.
# highest value of the prediction list is 0.22194855(at the last index 6) which is the index of disgust emotion
prediction[4]


array([0.11344204, 0.15389065, 0.14762376, 0.1799428 , 0.05881096,
       0.12434127, 0.22194855], dtype=float32)